In [1]:
%run init.ipynb

matchzoo version 1.0
`classification_task` initialized with metrics [accuracy]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ESIM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7907.59it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5599.55it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3324312.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11241.18it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5996.25it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1030633.86it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 296/296 [00:00<00:00, 11206.42it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2708/2708 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f57f2e69f60>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f57f2e47208>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='point'
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    mode='point'
)

In [7]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    sort=False,
    shuffle=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [8]:
model = mz.models.ESIM()

model.params['task'] = classification_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['dropout'] = 0.2
model.params['hidden_size'] = 200
model.params['lstm_layer'] = 1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(30058, 300)
  (rnn_dropout): RNNDropout(p=0.2, inplace=False)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2, inplace=False)
  )
  (out): Sequential(
    (0): Linear(in_features=200, out_features=2, bias=True)
    (1): Softmax(dim=-1)
  )
)
Trainable params:  9901402


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-1018 Loss-0.366]:
  Validation: accuracy: 0.9512



[Iter-2036 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-3054 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-4072 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-5090 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-6108 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-7126 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-8144 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-9162 Loss-0.364]:
  Validation: accuracy: 0.9512



[Iter-10180 Loss-0.364]:
  Validation: accuracy: 0.9512

Cost time: 3029.788060426712s
